In [25]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
#from langchain.vectorstores import Pinecone
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import os

In [2]:
# trỏ địa chỉ đến api_key
#pc = Pinecone(api_key="0dcca609-9f73-471b-9e57-882afb68c546")

# tạo index (đã tạo rồi nên skip)
#pc.create_index(name="cooking-chatbot", dimension=384, spec=ServerlessSpec(cloud='aws', region='us-east-1'))
#index = pc.Index("cooking-chatbot")

In [12]:
PINECONE_API_KEY = '0dcca609-9f73-471b-9e57-882afb68c546'

In [15]:
pc = Pinecone(api_key=PINECONE_API_KEY)
#pc.list_indexes().names() # check if index exsist
index_name='cooking-chatbot'
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [16]:
def load_data(data):
    loader = DirectoryLoader(data, glob='*.pdf', loader_cls=PyPDFLoader)

    documents = loader.load()
    return documents

In [17]:
extracted_data = load_data("data/")

Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 560 0 (offset 0)


In [18]:
# create text chucks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [19]:
text_chunks = text_split(extracted_data)
print("length of text chunks: ", len(text_chunks))

length of text chunks:  4624


In [20]:
def download_huggingface_embedding():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings             

In [21]:
embeddings = download_huggingface_embedding()

c:\Users\Lewis Do\AppData\Local\Programs\Python\Python39\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [22]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [23]:
query_result = embeddings.embed_query("Hello world")
print('Length', len(query_result))

Length 384


In [ ]:
query_result

[-0.03447723761200905,
 0.031023213639855385,
 0.006734990980476141,
 0.02610895223915577,
 -0.03936200216412544,
 -0.16030248999595642,
 0.06692393124103546,
 -0.006441502831876278,
 -0.04745049029588699,
 0.014758865348994732,
 0.07087529450654984,
 0.05552753433585167,
 0.019193345680832863,
 -0.026251327246427536,
 -0.010109513066709042,
 -0.026940496638417244,
 0.022307435050606728,
 -0.022226642817258835,
 -0.1496925801038742,
 -0.01749304123222828,
 0.007676258217543364,
 0.05435232073068619,
 0.0032544711139053106,
 0.031725890934467316,
 -0.0846213549375534,
 -0.029405983164906502,
 0.05159558728337288,
 0.04812406003475189,
 -0.0033148040529340506,
 -0.05827920883893967,
 0.04196924716234207,
 0.022210638970136642,
 0.1281888484954834,
 -0.022338991984725,
 -0.011656233109533787,
 0.06292837113142014,
 -0.03287629410624504,
 -0.09122603386640549,
 -0.031175388023257256,
 0.052699580788612366,
 0.0470348484814167,
 -0.08420310914516449,
 -0.030056176707148552,
 -0.020744847133

In [26]:
docs_chunks = [t.page_content for t in text_chunks]

vectorstore = PineconeVectorStore(index=index, pinecone_api_key=PINECONE_API_KEY, embedding=embeddings, namespace='cooking-chatbot',index_name='cooking-chatbot')

vectorstore.add_texts(texts=[t.page_content for t in text_chunks])

['558d715e-05cc-4e91-b131-b2e93457db97',
 'a7fc3ab9-31e9-4577-906a-f5b15b28e470',
 'e20e1cba-2378-411e-aac6-e4ead9f01c02',
 '4b795c7e-9db0-4a6d-828b-794c6403f718',
 '93fcd745-4604-4bc1-a156-f3d2793bde81',
 '84831eeb-a41f-412f-9615-cef21bd4ad40',
 'a2b03387-de8e-449f-aa67-0f0fe39066b7',
 '1320c7a0-c9b6-4643-9847-339e255fb55b',
 'cc17452f-e509-4540-81c5-9e393d189fe3',
 '9706f148-79c2-45bc-8150-20cf78fa5585',
 '65b82c76-06b7-47a6-8f81-a5261bc31447',
 '3a9a59cb-617f-47eb-a05f-284badb531e0',
 '27b6ed66-13a8-4ef2-8f43-dcc157464f16',
 '5a0bd9c1-d61c-4825-8188-5ae4ff080a7e',
 '6d597c2c-ca48-4907-91c9-23bbbfe0a2e6',
 'caef2790-d0b3-4f5d-b5fe-70c368c844f6',
 '79945197-8291-4ffe-b42a-9d46f72a8bd4',
 '6dacb827-141c-49e3-a564-7cab73c6f7a6',
 'f8152840-61ab-4b33-9cc1-de51a621e8ba',
 'ccb65f39-25d2-419b-9481-2a8ec1fe49f9',
 '34a2177b-3716-4c12-9e3d-03d11a685515',
 '5f67fb19-98be-4201-a503-aa0ac369ba85',
 '601ef99c-f85b-4905-81d6-90a10b55d6b1',
 '5b02760c-47a5-4081-8f81-152e60ad1adf',
 '67e6004b-efed-